# Week 3 Analysis Notebook

Goal now is to work towards combining output from last two weeks into making a prediction for the observed cosmic SED. Essentially means accouting for fact that higher $z$ galaxies will be dimmer. Aiming to do this through a lightcone using Chris Lovell's lightcone generation code for SIMBA. Then finally sum the apparent magnitudes as is already being done to get total SED.

In [ ]:
from pathlib import Path
import re
import numpy as np
import h5py

import os
os.environ['SPS_HOME'] = '/home/spujni/fsps'
import fsps

import caesar
import astropy.units as u
from astropy.constants import c
import matplotlib.pyplot as plt

import sys
# sys.path.insert(0, '../src')
# from week_2_funcs import get_redshift, list_snapshots, compute_summed_sed_from_appmags, compute_summed_sed_from_absmags, get_stellar_mass_bins, classify_galaxies
lightcone_path = Path('../src/lightcone').resolve()
sys.path.insert(0, str(lightcone_path.parent))
from lightcone.generate_lightcone import generate_lightcone, OUTPUT_DIR


CAT_DIR = Path("/home/spujni/sim/m50n512/s50/Groups/")

In [ ]:
# Generate a lightcone
lc_file = generate_lightcone(
    area_deg2=1.0,  # 1 square degree
    z_min=0.5,
    z_max=8.0,
    output_file=OUTPUT_DIR / "test_lightcone.h5",
    verbose=True
)

# Load and inspect
with h5py.File(lc_file, 'r') as f:
    print("Keys:", list(f.keys()))
    print("N galaxies:", f.attrs['n_galaxies'])
    z = f['z'][:]
    
plt.hist(z, bins=50)
plt.xlabel('Redshift')
plt.ylabel('N galaxies')
plt.title('Lightcone redshift distribution')
plt.show()

In [ ]:
from astropy.cosmology import Planck15 as cosmo

for z in [0.0, 0.1, 0.5, 1.0, 2.0, 5.0]:
    L_unit = cosmo.kpc_comoving_per_arcmin(z).to('Mpc / degree')
    A = (L_unit * 1.0**0.5).value  # 1 deg^2
    print(f"z={z}: 1 deg² = {A:.1f} Mpc, box is 50 Mpc, A > box? {A > 50.0}")

In [ ]:
# Load the lightcone
with h5py.File(OUTPUT_DIR / "test_lightcone.h5", 'r') as f:
    lc_z = f['z'][:]
    lc_snap = f['snap'][:]
    lc_idx = f['galaxy_index'][:]
    lc_stellar_mass = f['stellar_mass'][:]

print(f"Lightcone has {len(lc_z)} galaxies")

In [ ]:
# Function to compute apparent magnitude from absolute magnitude
def abs_to_app_mag(abs_mag, z):
    """Convert absolute magnitude to apparent magnitude at redshift z."""
    d_L = cosmo.luminosity_distance(z).to('pc').value  # in parsecs
    distance_modulus = 5 * np.log10(d_L / 10)  # 10 pc is the reference
    return abs_mag + distance_modulus


def flux_from_mag(mag):
    """Convert AB magnitude to flux (arbitrary units, but consistent)."""
    return 10**(-0.4 * mag)


def mag_from_flux(flux):
    """Convert flux back to AB magnitude."""
    return -2.5 * np.log10(flux)

In [ ]:
# Now loop through lightcone galaxies and compute SEDs
# Group by snapshot to avoid reloading Caesar files

from collections import defaultdict

# Group galaxies by snapshot
snap_groups = defaultdict(list)
for i, (snap, idx, z) in enumerate(zip(lc_snap, lc_idx, lc_z)):
    snap_groups[snap].append((i, idx, z))

print(f"Galaxies spread across {len(snap_groups)} snapshots")

In [ ]:
from astropy.cosmology import Planck15 as cosmo

# Initialize FSPS
sp = fsps.StellarPopulation(
    zcontinuous=1,
    sfh=0,  # SSP
    logzsol=0.0,
    dust_type=2,
)

# Get wavelength grid
wave = sp.wavelengths  # Angstroms

# Store all apparent-magnitude SEDs
all_app_seds = []  # List of (wavelength, flux) for each galaxy

for snap_num in sorted(snap_groups.keys()):
    galaxies_in_snap = snap_groups[snap_num]
    
    # Load Caesar catalogue for this snapshot
    cat_path = CAT_DIR / f"m50n512_{snap_num:03d}.hdf5"
    obj = caesar.load(str(cat_path))
    
    print(f"Processing snap {snap_num}: {len(galaxies_in_snap)} galaxies")
    
    for lc_i, gal_idx, gal_z in galaxies_in_snap:
        gal = obj.galaxies[gal_idx]
        
        # Get galaxy properties for FSPS
        stellar_mass = gal.masses['stellar'].value  # Solar masses
        metallicity = gal.metallicities['stellar']  # Mass-weighted Z
        
        # Simple approach: use mass-weighted age
        # (You might have a more sophisticated method from previous weeks)
        age_gyr = cosmo.age(gal_z).value  # Age of universe at this z
        
        # Set FSPS parameters
        sp.params['logzsol'] = np.log10(metallicity / 0.0142)  # Solar Z = 0.0142
        sp.params['tage'] = age_gyr
        
        # Get absolute magnitude SED (AB mags)
        mags_abs = sp.get_mags(bands=None)  # Returns spectrum in AB mags
        
        # Actually, get the spectrum in Lsun/Hz, then convert
        # sp.get_spectrum() returns (wave, spec) where spec is Lsun/Hz
        wave, spec_Lsun_Hz = sp.get_spectrum(tage=age_gyr)
        
        # Scale by stellar mass (FSPS assumes 1 Msun)
        spec_Lsun_Hz *= stellar_mass
        
        # Convert Lsun/Hz to flux at observer (erg/s/cm²/Hz)
        d_L = cosmo.luminosity_distance(gal_z).to('cm').value
        Lsun_erg_s = 3.828e33  # erg/s
        spec_flux = spec_Lsun_Hz * Lsun_erg_s / (4 * np.pi * d_L**2)
        
        # Apply redshift to wavelengths
        wave_observed = wave * (1 + gal_z)
        
        all_app_seds.append((wave_observed, spec_flux, gal_z))

print(f"Computed SEDs for {len(all_app_seds)} galaxies")

In [ ]:
# Now sum all SEDs onto a common wavelength grid
# Need to interpolate since each galaxy has different observed wavelengths

# Define common observed wavelength grid
wave_common = np.logspace(np.log10(1000), np.log10(50000), 500)  # 1000 Å to 5 μm

total_flux = np.zeros_like(wave_common)

for wave_obs, flux, z in all_app_seds:
    # Interpolate onto common grid
    flux_interp = np.interp(wave_common, wave_obs, flux, left=0, right=0)
    total_flux += flux_interp

# Convert to surface brightness (per steradian)
# 1 deg² = (π/180)² steradians
area_sr = (1.0 * np.pi / 180)**2  # 1 deg² in steradians
surface_brightness = total_flux / area_sr  # erg/s/cm²/Hz/sr

# Plot the cosmic SED
plt.figure(figsize=(10, 6))
plt.loglog(wave_common, surface_brightness * wave_common)  # νIν or λIλ
plt.xlabel('Observed Wavelength (Å)')
plt.ylabel(r'$\lambda I_\lambda$ (erg/s/cm²/sr)')
plt.title('Cosmic Optical/IR Background from Lightcone')
plt.xlim(1000, 50000)
plt.show()